In [ ]:
dataset_dir="/content/drive/MyDrive/Amazon/dataset"

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive 
import pandas as pd 
import csv
train_df = pd.read_csv(dataset_dir+'/train.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)
test_df = pd.read_csv(dataset_dir+'/test.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)
sample_df = pd.read_csv(dataset_dir+'/sample_submission.csv',escapechar = "\\",quoting = csv.QUOTE_NONE)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903024 entries, 0 to 2903023
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   TITLE           object
 1   DESCRIPTION     object
 2   BULLET_POINTS   object
 3   BRAND           object
 4   BROWSE_NODE_ID  int64 
dtypes: int64(1), object(4)
memory usage: 110.7+ MB


,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [ ]:
train_df['TITLE'].isna().sum()

71

In [ ]:
train_df_without_missing_values=train_df['TITLE'].dropna(how = 'all')

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!cp fasttext ../
%cd ..

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 29.87 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
/content/fastText
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/args.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/autotune.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/matrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/dictionary.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/loss.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/productquantizer.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/densematrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-loops -DNDEBUG -c src/quantmatrix.cc
c++ -pthread -std=c++11 -march=native -O3 -funroll-l

In [ ]:
train_df['TITLE'].to_csv('xgboost.txt', header=True, index=False, mode='a')

In [ ]:
!./fasttext skipgram -input xgboost.txt -output model


Read 42M words
Number of words:  220947
Number of labels: 0
Progress: 100.0% words/sec/thread:    7545 lr:  0.000000 avg.loss:  1.021679 ETA:   0h 0m 0s


In [ ]:
train_df_without_missing_values.to_csv('no_nan.txt', header=True, index=False, mode='a')

In [ ]:
!./fasttext skipgram -input no_nan.txt -output model_without_nan


Read 84M words
Number of words:  351659
Number of labels: 0
Progress:   2.1% words/sec/thread:    6964 lr:  0.048936 avg.loss:  1.531944 ETA:   1h22m21s^C


In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
custom_stop=['1','2','3','4','5','6','7','8']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
train_df_stop=train_df['TITLE'].str.lower().str.split()


In [ ]:
empty_list=[]
for i in train_df_stop:
  m_list=[]
  try:
    for j in i:
      if j not in stop:
        m_list.append(j)
    empty_list.append(m_list)
  except:
    pass


In [ ]:
with open('without_stopwords.txt', 'w') as f:
    for _list in empty_list:
        f.write(str(_list))

In [ ]:
new_df=train_df
new_df

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4
...,...,...,...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,Still want to use your favorite earphones/head...,"[Indian Connectors: Made for Indian sockets, t...",SHOPBELL,14790


In [ ]:
new_df.rename(columns={'BROWSE_NODE_ID': 'LABELS'},
          inplace=True, errors='raise')

In [ ]:
new_df=new_df[['TITLE','LABELS']]

In [ ]:
new_df.to_csv('supervised_input.txt', header=True, index=False, mode='a')

In [ ]:
!./fasttext supervised -input supervised_input.txt -label __label__ -output model

Read 126M words
Number of words:  3663403
Number of labels: 0
tcmalloc: large alloc 1465368576 bytes == 0x55c6bfb80000 @  0x7f69b9da0887 0x55c683dc9fed 0x55c683dd871e 0x55c683de04fc 0x55c683de7ffc 0x55c683d9d887 0x7f69b8e3dbf7 0x55c683d9db4a
Progress: 100.0% words/sec/thread:  645830 lr:  0.000000 avg.loss:      -nan ETA:   0h 0m 0s
^C


In [ ]:
new_df['LABELS']='__label__' + new_df['LABELS'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
new_df

,TITLE,LABELS
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",__label__0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",__label__1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,__label__2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),__label__3
4,Teacher Planner Company A4 6 Lesson Academic T...,__label__4
...,...,...
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,__label__1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,__label__15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,__label__1044933
2903022,Type-C to 3.5 MM for Oppo R17 Pro Type-C to 3....,__label__14790


In [ ]:
!./fasttext skipgram  -input without_stopwords.txt -epoch 10  -output model_without_stopwords


Read 33M words
Number of words:  162880
Number of labels: 0
Progress: 100.0% words/sec/thread:    4645 lr:  0.000000 avg.loss:  0.793510 ETA:   0h 0m 0s


In [ ]:
!./fasttext skipgram -input /content/drive/MyDrive/Amazon/file1.txt -epoch 13 -output best_embeddings_withoutstop


Read 33M words
Number of words:  150988
Number of labels: 0
Progress: 100.0% words/sec/thread:    5089 lr:  0.000000 avg.loss:  0.718742 ETA:   0h 0m 0s


In [ ]:
x_df= train_df[['TITLE','BROWSE_NODE_ID']]

In [ ]:
x_df.to_csv('queries.txt')

 ./fasttext print-sentence-vectors model.bin < text.txt

In [ ]:
 !./fasttext print-sentence-vectors best_embeddings_withoutstop.bin < queries.txt >out.txt

In [ ]:
f=open('/content/out.txt',mode='r')
#for i in f:
  #print(i)

import pandas as pd
df = pd.read_csv('out.txt')
print(df)

        0.10108 0.18006 -0.30638 -0.096125 0.028631 -0.073578 -0.052795 -0.062455 0.038677 0.034355 0.054682 0.0039096 -0.06137 0.13893 0.024815 -0.022108 0.12296 0.0027795 0.10168 0.27184 -0.12972 0.01024 -0.10889 0.077532 0.012771 0.13756 -0.003516 0.039824 0.12883 -0.14598 0.013654 -0.17926 0.018531 0.1235 -0.048784 -0.019624 0.038161 0.10675 -0.0038568 0.06317 -0.0083131 0.013357 0.081056 -0.023178 0.17597 0.23725 -0.055782 -0.030217 0.04228 0.15887 -0.012786 0.075506 -0.074808 -0.035776 0.18746 -0.078033 -0.12349 0.038912 0.077493 -0.19037 0.082762 -0.02742 0.046501 -0.011316 0.073546 0.017981 0.041072 -0.049389 0.096939 -0.072759 -0.12968 0.034054 0.054838 0.03369 0.096233 -0.01669 -0.081715 0.0084669 0.18319 -0.084174 -0.084507 -0.01311 -0.11857 -0.031227 0.019685 0.17661 -0.035458 -0.054512 -0.1165 0.12051 -0.11764 0.094507 0.11133 0.11613 -0.1037 -0.040763 -0.16408 0.10094 -0.047747 0.11111 
0        0.01607 0.064157 -0.084405 0.058051 0.025298 0...                            

In [ ]:
 result = pd.concat([df, x_df], axis=1, join="inner")

In [ ]:
result.to_csv('csv.csv')

In [ ]:
!mv /content/csv.csv /content/drive/MyDrive/Amazon

In [ ]:
!zip /content/drive/MyDrive/Amazon/csv.zip /content/drive/MyDrive/Amazon/csv.csv 

  adding: content/drive/MyDrive/Amazon/csv.csv (deflated 62%)


In [ ]:
!mv /content/best_embeddings_withoutstop.bin /content/drive/MyDrive/Amazon

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
path='/content/drive/MyDrive/Amazon/csv.csv'
f=open(path,mode='r')
df=pd.read_csv(f,index_col=None)


In [ ]:
df.rename(columns = {'0.10108 0.18006 -0.30638 -0.096125 0.028631 -0.073578 -0.052795 -0.062455 0.038677 0.034355 0.054682 0.0039096 -0.06137 0.13893 0.024815 -0.022108 0.12296 0.0027795 0.10168 0.27184 -0.12972 0.01024 -0.10889 0.077532 0.012771 0.13756 -0.003516 0.039824 0.12883 -0.14598 0.013654 -0.17926 0.018531 0.1235 -0.048784 -0.019624 0.038161 0.10675 -0.0038568 0.06317 -0.0083131 0.013357 0.081056 -0.023178 0.17597 0.23725 -0.055782 -0.030217 0.04228 0.15887 -0.012786 0.075506 -0.074808 -0.035776 0.18746 -0.078033 -0.12349 0.038912 0.077493 -0.19037 0.082762 -0.02742 0.046501 -0.011316 0.073546 0.017981 0.041072 -0.049389 0.096939 -0.072759 -0.12968 0.034054 0.054838 0.03369 0.096233 -0.01669 -0.081715 0.0084669 0.18319 -0.084174 -0.084507 -0.01311 -0.11857 -0.031227 0.019685 0.17661 -0.035458 -0.054512 -0.1165 0.12051 -0.11764 0.094507 0.11133 0.11613 -0.1037 -0.040763 -0.16408 0.10094 -0.047747 0.11111 ':'Embeddings'},inplace=True)

In [ ]:
dataframe=pd.DataFrame()
for i in df['Embeddings']:
  dataframe.concat(i.split(' '))

AttributeError: ignored